In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import sys
from html.parser import HTMLParser
from html.entities import name2codepoint
sns.set(color_codes=True)
import warnings
warnings.filterwarnings("ignore")                   
import nltk                                         
nltk.download('stopwords')
from nltk.corpus import stopwords                   
from nltk.stem import PorterStemmer  
LA = np.linalg

from sklearn.feature_extraction.text import CountVectorizer          
from sklearn.feature_extraction.text import TfidfVectorizer   
from sklearn.feature_extraction.text import HashingVectorizer
from scipy.sparse import csr_matrix
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from collections import defaultdict
#from gensim.models.word2vec import Word2Vec  
import spacy
from spacy.lang.en.examples import sentences

import re

%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/chd415/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load training data text
def load_data(filename):
    load_file = pd.read_csv(filename,delimiter=',', header=0,
                        dtype={'name':str, 'lvl1':str, 'lvl2':str, 'lvl3':str, 'descrption':str, 'type':str})
    load_file.columns = ['id', 'name','lvl1','lvl2','lvl3','descrption','price','type']
    load_file.duplicated(subset=None, keep='first')
    load_file.set_index('id', inplace = True)
    load_file.head()
    return load_file
#print(len(train_file))
def load_label(filename):
    load_label = pd.read_csv(filename,delimiter=',', header=0)
    load_label.columns = ['id', 'score']
    load_label.duplicated(subset=None, keep='first')
    load_label.set_index('id', inplace = True)
    return load_label

In [3]:
class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)


In [ ]:
'''
def text_embedding(column,vecsize):
    temp_X = column.astype(str)
    stop = set(stopwords.words('english'))
    temp =[]
    snow = nltk.stem.SnowballStemmer('english')

    for sentence in temp_X: 
        words = [snow.stem(word) for word in sentence.split(' ') if word not in stopwords.words('english')]   # Stemming and removing stopwords
        temp.append(sentence)

    count_vect = CountVectorizer(max_features=10000)
    bow_data = count_vect.fit_transform(temp)

    final_tf = temp
    tf_idf = TfidfVectorizer(max_features=10000)
    tf_data = tf_idf.fit_transform(final_tf)
    w2v_data = temp
    splitted = []
    for row in w2v_data: 
        splitted.append([word for word in row.split()])     #splitting words
    
    train_w2v = Word2Vec(splitted,min_count=1,size=vecsize, workers=4)
    
    avg_data = []
    for row in splitted:
        vec = np.zeros(vecsize, dtype=float)
        count = 0
        for word in row:
            try:
                vec += train_w2v[word]
                count += 1
            except:
                pass
        if (count == 0):
            avg_data.append(vec)
        else:
            avg_data.append(vec/count)

        
    tf_w_data = []
    tf_data = tf_data.toarray()
    i = 0
    for row in splitted:
        vec = [0 for i in range(vecsize)]
    
        temp_tfidf = []
        for val in tf_data[i]:
            if val != 0:
                temp_tfidf.append(val)
    
        count = 0
        tf_idf_sum = 0
        for word in row:
            try:
                count += 1
                tf_idf_sum = tf_idf_sum + temp_tfidf[count-1]
                vec += (temp_tfidf[count-1] * train_w2v[word])
            except:
                pass
        if (tf_idf_sum == 0):
            tf_w_data.append(vec)
        else:
            tf_w_data.append(vec/tf_idf_sum)
        i = i + 1
    
    return tf_w_data
'''

In [ ]:
'''
#test cell
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import Normalizer

temp = load_data('train_data.csv')
price_X = temp['price'].as_matrix(columns=None).reshape(-1, 1)

# scaler = StandardScaler()
# scaler.fit(price_X)
# outfile = scaler.transform(price_X)

# est = KBinsDiscretizer(n_bins=5, encode='onehot-dense')
# outfile = est.fit_transform(price_X)

# transformer = Normalizer(norm='l2').fit(price_X)
# outfile = transformer.transform(price_X)


# temp['price'] = temp['price']
valuethred = 5000.
temp['price'] = temp['price'].clip(lower=0.,upper=valuethred)
hist = temp['price'].hist(bins=20)

# price_X = temp['price'].clip(lower=0.).as_matrix(columns=None).tolist()
# price_X.sort()

#np.histogram(outfile,bins=10)

# outfilelist = outfile.tolist()
#outfilelist.sort()

# file = open('temp.dat','w')
# file.writelines(["%s\n" % item  for item in outfilelist])
# file.close()
#print(outfile)
'''

In [ ]:
'''
#test cell
#test pretrained word2vec

import spacy
from spacy.lang.en.examples import sentences

temp = pd.read_csv('train_data.csv',delimiter=',', header=0, nrows=10)
description_X = temp.descrption.str.lower().replace('<.*?>','',regex=True).replace('\d+', '',regex=True)
descrption_Xstring = pd.Series(description_X.tolist()).astype(str)

clg = spacy.load('en_core_web_lg')
descrption_clg = []
for i in range(len(descrption_Xstring)):
    doc = clg(descrption_Xstring[i])
    descrption_clg.append(doc.vector)
descrption_clg_array = np.vstack(descrption_clg)

cmd = spacy.load('en_core_web_md')
descrption_cmd = []
for i in range(len(descrption_Xstring)):
    doc = cmd(descrption_Xstring[i])
    descrption_cmd.append(doc.vector)
descrption_cmd_array = np.vstack(descrption_cmd)

vlg = spacy.load('en_vectors_web_lg')
descrption_vlg = []
for i in range(len(descrption_Xstring)):
    doc = vlg(descrption_Xstring[i])
    descrption_vlg.append(doc.vector)
descrption_vlg_array = np.vstack(descrption_vlg)

tfidf_h = HashingVectorizer(n_features=2**10,stop_words = 'english')
lh = tfidf_h.fit_transform(description_X.astype('U')).toarray()

tfidf_v = CountVectorizer(max_features=500,stop_words = 'english')
lv = tfidf_v.fit_transform(description_X.astype('U')).toarray()

tfidf_t = TfidfVectorizer(max_features=500,stop_words = 'english')
lt = tfidf_t.fit_transform(description_X.astype('U')).toarray()



print(descrption_new_array)
print(descrption_cmd_array)
print(descrption_vlg_array)
print(lh)
print(lv)
print(lt)
'''

In [ ]:
'''
#test cell
#one-hot for lel 1-3&region&text-counts backup

def clean_data(filename):
    
    filename['lvl1'] = filename['lvl1'].str.lower().replace('[^a-zA-Z]+',' ',regex=True)
    filename['lvl2'] = filename['lvl2'].str.lower().replace('[^\'\w]+',' ',regex=True)
    filename['lvl3'] = filename['lvl3'].str.lower().replace('[^\'\w]+',' ',regex=True)
    filename['descrption'] = filename['descrption'].str.lower()
    filename['name'] = filename['name'].str.lower()

    #clean up data for lvl 1&2&3 and type
    temp =  filename.drop(['price', 'descrption','name'], axis=1)
    outfile = MultiColumnLabelEncoder(columns = ['lvl1','lvl2','lvl3','type']).fit_transform(temp.astype(str))
    
    #count text information
    description_X = filename.descrption.str.lower().replace('<li>','final ',regex=True).replace('<.*?>','',regex=True).replace('[^\w\s]+','',regex=True).replace('\d+','',regex=True)
    outfile['des_bullitcount'] = description_X.str.count('final').fillna(0)
    description2count = description_X.replace('final ','',regex=True)
    descrption_Xstring = pd.Series(description2count.tolist()).astype(str)
    outfile['des_wordcount'] = descrption_Xstring.apply(lambda x: len(x.split(' '))).fillna(0)
    outfile['des_lettercount'] = descrption_Xstring.apply(lambda x: len(x)).fillna(0)
    outfile.des_wordcount.iloc[-1] = 21.
    outfile.des_lettercount.iloc[-1] = 92.

    name2count = filename.name.str.lower().replace('<.*?>','',regex=True).replace('[^\w\s]+',' ',regex=True).replace('\d+', '',regex=True)
    name_Xstring = pd.Series(name2count.tolist()).astype(str)
    outfile['name_wordcount'] = name_Xstring.apply(lambda x: len(x.split(' '))).fillna(0)
    outfile['name_lettercount'] = name_Xstring.apply(lambda x: len(x)).fillna(0)
    outfile.name_wordcount.iloc[-1] = 6.
    outfile.name_lettercount.iloc[-1] = 31.
    
    #one-hot encoder
    enc = preprocessing.OneHotEncoder()
    enc.fit(outfile)
    outfile = enc.transform(outfile).toarray()
    
    
    
    #normalize price

    price_X = filename['price'].as_matrix(columns=None).reshape(-1, 1)
    transformer = Normalizer(copy=True,norm='l2')
    ld = transformer.fit_transform(price_X)
    outfile = np.column_stack((outfile,ld))

    

    #clean up text

    description_X = filename.descrption.str.lower().replace('<.*?>','',regex=True).replace('[^\w\s]+',' ',regex=True).replace('\d+','',regex=True)
    
    tfidf_n = HashingVectorizer(n_features=2**10,stop_words = 'english')
    lg = tfidf_n.fit_transform(description_X.astype('U')).toarray()
    
    name_X = filename.name.str.lower().replace('<.*?>','',regex=True).replace('[^\w\s]+',' ',regex=True).replace('\d+','',regex=True)

    tfidf_f = HashingVectorizer(n_features=2**10,stop_words = 'english')
    lf = tfidf_f.fit_transform(name_X.astype('U')).toarray()
   
    svd_n = TruncatedSVD(n_components=30,algorithm='arpack')
    new_lg = svd_n.fit_transform(lg.astype('f'))
    svd_f = TruncatedSVD(n_components=15,algorithm='arpack')
    new_lf = svd_n.fit_transform(lf.astype('f'))
    
    outfile = np.column_stack((outfile,new_lg))
    outfile = np.column_stack((outfile,new_lf))
    
    
    return outfile


'''

In [4]:
def clean_data(filename):
    
    filename['lvl1'] = filename['lvl1'].str.lower().replace('[^a-zA-Z]+',' ',regex=True)
    filename['lvl2'] = filename['lvl2'].str.lower().replace('[^\'\w]+',' ',regex=True)
    filename['lvl3'] = filename['lvl3'].str.lower().replace('[^\'\w]+',' ',regex=True)
    filename['descrption'] = filename['descrption'].str.lower()
    filename['name'] = filename['name'].str.lower()

    #clean up data for lvl 1&2&3 and type
    temp =  filename.drop(['price', 'descrption','name'], axis=1)
    outfile = MultiColumnLabelEncoder(columns = ['lvl1','lvl2','lvl3','type']).fit_transform(temp.astype(str))
    
    #one-hot encoder
    enc = preprocessing.OneHotEncoder()
    enc.fit(outfile)
    outfile = enc.transform(outfile).toarray()

    #bin price
    valuethred = 5000.
    clipped_price = filename['price'].clip(lower=0.,upper=valuethred)
    price_kbin = clipped_price.as_matrix(columns=None).reshape(-1, 1)
    kbinest = KBinsDiscretizer(n_bins=20, encode='onehot-dense')
    kbinout = kbinest.fit_transform(price_kbin)
    outfile = np.column_stack((outfile,kbinout))

    #normalize price
    valuethred = 5000.
    clipped2norm = filename['price'].clip(lower=0.,upper=valuethred)
    price_X = clipped2norm.as_matrix(columns=None).reshape(-1, 1)
    transformer = Normalizer(copy=True,norm='l2')
    ld = transformer.fit_transform(price_X)
    outfile = np.column_stack((outfile,ld))  

    #clean up text
    description_X = filename.descrption.str.lower().replace('<li>','final ',regex=True).replace('<.*?>','',regex=True).replace('[^\w\s]+',' ',regex=True)
    count_descrption = description_X.str.count('final').fillna(0).tolist()
    outfile = np.column_stack((outfile,count_descrption))
    
    description_X = description_X.str.lower().replace('final ','',regex=True).replace('\d+', '',regex=True)
    descrption_Xstring = pd.Series(description_X.tolist()).astype(str)
    count_wordcount = descrption_Xstring.apply(lambda x: len(x.split(' ')))
    count_lettercount = descrption_Xstring.apply(lambda x: len(x))
    outfile = np.column_stack((outfile,count_wordcount))
    outfile = np.column_stack((outfile,count_lettercount))
    
#     tfidf_n = HashingVectorizer(n_features=2**10,stop_words = 'english')
#     #tfidf_n = TfidfVectorizer(max_features=500,stop_words = 'english')
#     #tfidf_n = CountVectorizer(max_features=500,stop_words = 'english')
#     lg = tfidf_n.fit_transform(description_X.astype('U')).toarray()

    clg_g = spacy.load('en_core_web_lg')
    descrption_clg = []
    for i in range(len(descrption_Xstring)):
        doc = clg_g(descrption_Xstring[i])
        descrption_clg.append(doc.vector)
    lg = np.vstack(descrption_clg)
    
    
    name_X = filename.name.str.lower().replace('<.*?>','',regex=True).replace('[^\w\s]+',' ',regex=True).replace('\d+', '',regex=True)
    name_Xstring = pd.Series(name_X.tolist()).astype(str)
    name_wordcount = name_Xstring.apply(lambda x: len(x.split(' ')))
    name_lettercount = name_Xstring.apply(lambda x: len(x))
    outfile = np.column_stack((outfile,name_wordcount))
    outfile = np.column_stack((outfile,name_lettercount))


#     tfidf_f = HashingVectorizer(n_features=2**10,stop_words = 'english')
#     #tfidf_f = TfidfVectorizer(max_features=500,stop_words = 'english')
#     #tfidf_f = CountVectorizer(max_features=500,stop_words = 'english')
#     lf = tfidf_f.fit_transform(name_X.astype('U')).toarray()
    
    clg_f = spacy.load('en_core_web_lg')
    name_clg = []
    for i in range(len(name_Xstring)):
        doc = clg_f(name_Xstring[i])
        name_clg.append(doc.vector)
    lf = np.vstack(name_clg)

    #svd
    svd_n = TruncatedSVD(n_components=30,algorithm='arpack')
    new_lg = svd_n.fit_transform(lg.astype('f'))
    svd_f = TruncatedSVD(n_components=15,algorithm='arpack')
    new_lf = svd_n.fit_transform(lf.astype('f'))
    
    outfile = np.column_stack((outfile,new_lg))
    outfile = np.column_stack((outfile,new_lf))
    
    return outfile

In [ ]:
'''
train_file = pd.read_csv('train_data.csv',delimiter=',', header=0, nrows=10)
train_file.columns = ['id', 'name','lvl1','lvl2','lvl3','descrption','price','type']
description_X = train_file.descrption.str.lower().replace('<li>','final ',regex=True).replace('<.*?>','',regex=True).replace('[^\w\s]+',' ',regex=True)
count_descrption = description_X.str.count('final')
description_X = train_file.descrption.str.lower().replace('final ','',regex=True)
print(np.shape(count_descrption))

'''


In [5]:
train_file = load_data('train_data.csv')
test_file = load_data('test_data.csv')
combined_file = pd.concat([train_file,test_file])
cleaned_train = clean_data(combined_file)
train_score = load_label('train_label.csv')
np.shape(cleaned_train)

(36283, 340)

In [ ]:
'''
s_gra = np.abs(np.gradient(s))
slist = s.tolist()
file = open('svd_secrption.dat','w')
file.writelines(["%s\n" % item  for item in s_gra])
file.close()

import matplotlib.pyplot as plt
fig, ax1 = plt.subplots()
color = 'tab:red'
ax1.set_ylabel('Eign Values', color=color,fontweight='bold')
ax1.plot(slist, color=color)
ax1.tick_params(axis='y', labelcolor=color)
ax1.set_xscale('log')


ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
color = 'tab:blue'
ax2.set_ylabel('Gradient of Eign Value', color=color,fontweight='bold')  # we already handled the x-label with ax1
ax2.plot(s_gra, color=color)
ax2.tick_params(axis='y', labelcolor=color)
ax2.set_xscale('log')
ax2.set_yscale('log')
plt.grid()
fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

fig.savefig('Eign_Values_descrption.png', dpi=fig.dpi)
'''

In [6]:
'''
# test cell
cleaned_train['descrption'] = temp['descrption']
'''
cleaned_train

array([[ 0.        ,  0.        ,  0.        , ...,  0.34309661,
         0.17233208,  0.45813119],
       [ 0.        ,  1.        ,  0.        , ..., -0.04276813,
         0.20824412, -0.23762988],
       [ 0.        ,  0.        ,  0.        , ..., -0.16593644,
        -0.04395773,  0.114619  ],
       ...,
       [ 0.        ,  0.        ,  0.        , ..., -0.14239173,
        -0.00520656, -0.13928017],
       [ 0.        ,  0.        ,  0.        , ..., -0.04370552,
         0.08029149, -0.26857373],
       [ 0.        ,  0.        ,  0.        , ...,  0.10217056,
        -0.12637366,  0.1478506 ]])

In [ ]:
'''
pca = PCA(n_components=100, svd_solver='full')
cleaned_train_temp = pca.fit_transform(cleaned_train)

cleaned_train_min = np.min(cleaned_train_temp)
cleaned_train_out = cleaned_train_temp - (cleaned_train_min)*np.ones_like(cleaned_train_temp.size)
print(cleaned_train_out)
cleaned_train = cleaned_train_out
print(cleaned_train.shape)
'''

In [7]:
'''

def rearrange(cleaned_data):
    la = cleaned_data.lvl1.as_matrix(columns=None).tolist()
    lb = cleaned_data.lvl2.as_matrix(columns=None).tolist()
    lc = cleaned_data.lvl3.as_matrix(columns=None).tolist()

    X = la
    X = np.column_stack((X,lb))
    X = np.column_stack((X,lc))

    enc = preprocessing.OneHotEncoder()
    enc.fit(X)
    X = enc.transform(X).toarray()
    
    ld = cleaned_data.price.as_matrix(columns=None).tolist()
    le = cleaned_data.type.as_matrix(columns=None).tolist()

    X = np.column_stack((X,ld))
    X = np.column_stack((X,le))

   
    lf = cleaned_data.name.as_matrix(columns=None).tolist()
    lg = cleaned_data.descrption.as_matrix(columns=None).tolist()
    lg_array = np.vstack( lg )
    lf_array = np.vstack( lf )

    U, s, Vh = LA.svd(lg_array, full_matrices=False)
    assert np.allclose(lg_array, np.dot(U, np.dot(np.diag(s), Vh)))

# only use U \cdot s
    
#    s[8:] = 0.
#    new_lg = np.dot(U, np.dot(np.diag(s), Vh))
    s_new = s[:5]
    U_new = U[:, :5]
    new_lg = np.dot(U_new, np.diag(s_new))
    lg_min = np.min(new_lg)
    lg_out = new_lg  - (lg_min-2)*np.ones_like(new_lg.size)

    Uf, sf, Vhf = LA.svd(lf_array, full_matrices=False)
    assert np.allclose(lf_array, np.dot(Uf, np.dot(np.diag(sf), Vhf)))

#    sf[5:] = 0.
#    new_lf = np.dot(Uf, np.dot(np.diag(sf), Vhf))
    sf_new = sf[:5]
    Uf_new = Uf[:, :5]
    new_lf = np.dot(Uf_new, np.diag(sf_new))
    lf_min = np.min(new_lf)
    lf_out = new_lf  - (lf_min-2)*np.ones_like(new_lf.size)

    
    X = np.column_stack((X,lf_out))
    X = np.column_stack((X,lg_out))
    X = X.tolist()
   
    return X,lf_min,lg_min
'''
    
    #print(len(X))
X = cleaned_train#rearrange(cleaned_train)
w,b = np.shape(np.array(X))
print(np.shape(np.array(X)))
Y = train_score.score.as_matrix(columns=None).tolist()
b

(36283, 340)


340

In [8]:
X = cleaned_train[:18141]
XX = cleaned_train[18141:]
Y = train_score.score.as_matrix(columns=None).tolist()
print(np.size(Y))
print(np.shape(X))
print(np.shape(XX))


18141
(18141, 340)
(18142, 340)


In [9]:
from sklearn.utils import shuffle
X, Y = shuffle(X, Y)
from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(X, Y, test_size=0.20, random_state=0)


#maxlen = b
#X_train = sequence.pad_sequences(X_train, maxlen=maxlen, dtype='float32')
#X_validation = sequence.pad_sequences(X_validation, maxlen=maxlen, dtype='float32')
print(X_train[1400].size)

340


In [10]:
import os
import time
from sklearn import metrics
from sklearn import preprocessing
import numpy as np
import pandas as pd
import random
import math
import warnings
warnings.filterwarnings("ignore")

from matplotlib import pyplot as plt

from sklearn import preprocessing
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import chi2

from sklearn import linear_model
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor 
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
#from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.ensemble import BaggingClassifier  
from sklearn.model_selection import cross_val_score

In [11]:
# Linear Regression Classifier
def linear_regression_classifier(train_x, train_y):
    model = linear_model.LinearRegression()
    model.fit(train_x, train_y)
    return model
 
# Multinomial Naive Bayes Classifier
def naive_bayes_classifier(train_x, train_y):
    model = MultinomialNB()

    param_grid = {'alpha': [math.pow(10,-i) for i in range(11)]}
    grid_search = GridSearchCV(model, param_grid, n_jobs = 1, verbose=1)
    grid_search.fit(train_x, train_y)
    best_parameters = grid_search.best_estimator_.get_params()
    
    model = MultinomialNB(alpha = best_parameters['alpha'])  
    model.fit(train_x, train_y)
    return model
 
 
# KNN Classifier
def knn_classifier(train_x, train_y):
    model = KNeighborsClassifier()

    param_grid = {'n_neighbors': list(range(1,21))}
    grid_search = GridSearchCV(model, param_grid, n_jobs = 1, verbose=1)
    grid_search.fit(train_x, train_y)
    best_parameters = grid_search.best_estimator_.get_params()
    
    model = KNeighborsClassifier(n_neighbors = best_parameters['n_neighbors'], algorithm='kd_tree')

    bagging = BaggingClassifier(model, max_samples=0.5, max_features=1 )
    bagging.fit(train_x, train_y)
    return bagging
 
 
# Logistic Regression Classifier
def logistic_regression_classifier(train_x, train_y):
    model = LogisticRegression(penalty='l2')
    model.fit(train_x, train_y)
    return model
 
 
# Random Forest Classifier
def random_forest_classifier(train_x, train_y):
    model = RandomForestClassifier()

    param_grid = {'n_estimators': list(range(1,21))}
    grid_search = GridSearchCV(model, param_grid, n_jobs = 1, verbose=1)
    grid_search.fit(train_x, train_y)
    best_parameters = grid_search.best_estimator_.get_params()
    
    model = RandomForestClassifier(n_estimators = best_parameters['n_estimators'])
    
    model.fit(train_x, train_y)
    return model
 
 
# Decision Tree Classifier
def decision_tree_classifier(train_x, train_y):
    model = tree.DecisionTreeClassifier()
    model.fit(train_x, train_y)

    bagging = BaggingClassifier(model, max_samples=0.5, max_features=1 )
    bagging.fit(train_x, train_y)
    
    return bagging
 
 
# GBDT(Gradient Boosting Decision Tree) Classifier
def gradient_boosting_classifier(train_x, train_y):
    model = GradientBoostingClassifier()
    
    model = RandomForestClassifier()

    param_grid = {'n_estimators': list(range(100,300,10))}
    grid_search = GridSearchCV(model, param_grid, n_jobs = 1, verbose=1)
    grid_search.fit(train_x, train_y)
    best_parameters = grid_search.best_estimator_.get_params()
    
    model = RandomForestClassifier(n_estimators = best_parameters['n_estimators'])

    model.fit(train_x, train_y)
    return model

# SVM Classifier
def svm_classifier(train_x, train_y):
    model = SVC(kernel='linear', probability=True)
    model.fit(train_x, train_y)
    return model
 
# SVM Classifier using cross validation
def svm_cross_validation(train_x, train_y):
    model = SVC(kernel='linear', probability=True)
    param_grid = {'C': [1e-3, 1e-2, 1e-1, 1, 10, 100, 1000], 'gamma': [0.001, 0.0001]}
    grid_search = GridSearchCV(model, param_grid, n_jobs = 1, verbose=1)
    grid_search.fit(train_x, train_y)
    best_parameters = grid_search.best_estimator_.get_params()
    #for para, val in best_parameters.items():
        #print para, val
    model = SVC(kernel='rbf', C=best_parameters['C'], gamma=best_parameters['gamma'], probability=True)
    model.fit(train_x, train_y)
    return model

def feature_select(x,y):
    clf = ExtraTreesClassifier()
    clf = clf.fit(x, y)
    model = SelectFromModel(clf, prefit=True)
    return model

In [12]:
# just for my own record


if __name__ == '__main__': 
    thresh = 0.5    
    
    test_classifiers = ['LR','RF','DT']    
    classifiers = {
                    'LR':logistic_regression_classifier,
                    'RF':random_forest_classifier,
                    'DT':decision_tree_classifier,
    }
    
    '''
    test_classifiers = ['SVC']    
    classifiers = {'SVC':svm_cross_validation}
    ''' 
    
    print('reading training and testing data...')    

    select_model = feature_select(X_train, y_train)
    X_train = select_model.transform(X_train)
    X_validation = select_model.transform(X_validation)

    result = []
    
    '''
    start_time = time.time()    
    model = classifiers[classifier](X_train, y_train)   
    print('training took %fs!' % (time.time() - start_time))    
    regressor = DecisionTreeRegressor()
    regressor.fit(train_x, train_y)
    predict = model.predict(X_validation)
    '''

        
    for classifier in test_classifiers:    
        print('******************* %s ********************' % classifier)    
        start_time = time.time()    
        model = classifiers[classifier](X_train, y_train)   
        print('training took %fs!' % (time.time() - start_time))    
        predict = model.predict(X_validation)

        precision = metrics.precision_score(y_validation, predict)    
        recall = metrics.recall_score(y_validation, predict)    
        accuracy = metrics.accuracy_score(y_validation, predict)    
        print('accuracy: %.2f%%' % (100 * accuracy))
        logloss = metrics.log_loss(y_validation, predict)
        print('loss: %.2f' % (logloss))

        scores = cross_val_score(model, X_train, y_train)
#        print(scores)

reading training and testing data...
******************* LR ********************
training took 1.214134s!
accuracy: 80.46%
loss: 6.75
******************* RF ********************
Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   39.3s finished


training took 43.186757s!
accuracy: 80.99%
loss: 6.57
******************* DT ********************
training took 1.909744s!
accuracy: 67.29%
loss: 11.30


In [13]:
X_train
#X_train, y_train = X,Y

array([[ 0.        ,  0.        ,  0.        , ..., -0.08697326,
        -0.18759921,  0.11283299],
       [ 0.        ,  1.        ,  0.        , ...,  0.37877461,
         0.35219729,  0.17430373],
       [ 0.        ,  0.        ,  1.        , ..., -0.0323269 ,
        -0.10206458,  0.20948608],
       ...,
       [ 0.        ,  0.        ,  1.        , ...,  0.09015514,
         0.12230652,  0.12184646],
       [ 0.        ,  0.        ,  1.        , ...,  0.07511818,
         0.16912353,  0.06112052],
       [ 0.        ,  0.        ,  1.        , ..., -0.0028989 ,
        -0.36047363,  0.35635692]])

In [ ]:
#test cnn model

from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Conv1D, MaxPooling1D

# Embedding
max_features = 5000
maxlen = b
embedding_size = 128

# Convolution
kernel_size = 5
filters = 32
pool_size = 3

# LSTM
lstm_output_size = 70

# Training
batch_size = 30
epochs = 5

'''
Note:
batch_size is highly sensitive.
Only 2 epochs are needed as the dataset is very small.
'''

print('Loading data...')
X_train = np.asarray(np.abs(X_train))
X_validation = np.asarray(np.abs(X_validation))
print(len(X_train), 'train sequences')
print(len(X_validation), 'test sequences')

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=maxlen, padding='post')
X_validation = sequence.pad_sequences(X_validation, maxlen=maxlen, padding='post')
print('x_train shape:', X_train.shape)
print('x_test shape:', X_validation.shape)

print('Build model...')

cnnmodel = Sequential()
cnnmodel.add(Embedding(max_features, embedding_size,input_length=maxlen))
#model.add(Dense(32, activation='relu', input_dim=100))
cnnmodel.add(Dropout(0.5))
cnnmodel.add(Dense(32, activation='relu', input_dim=64))
cnnmodel.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
cnnmodel.add(MaxPooling1D(pool_size=pool_size))
cnnmodel.add(LSTM(lstm_output_size))
cnnmodel.add(Dense(1))
cnnmodel.add(Activation('sigmoid'))

cnnmodel.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [14]:
X_train = cleaned_train[:18141]
X_test = cleaned_train[18141:]
y_train = train_score.score.as_matrix(columns=None).tolist()

In [15]:
if __name__ == '__main__': 
    thresh = 0.5    
     
    test_classifiers = ['LR','RF','GBC','DT']    
    classifiers = {
                   'LR':logistic_regression_classifier,
                   'RF':random_forest_classifier,
                   'GBC':gradient_boosting_classifier,
                   'DT':decision_tree_classifier
    }
        
    print('reading training and testing data...')    

    select_model = feature_select(X_train, y_train)
    X_train = select_model.transform(X_train)
    X_test = select_model.transform(X_test)

    result = []
    
    start_time = time.time()
    modellr = classifiers['LR'](X_train, y_train)   
    print('training took %fs!' % (time.time() - start_time))    
    Y_predict_lr = modellr.predict_proba(X_test)[:,1]
    print('predict finished')
    
    modelrf = classifiers['RF'](X_train, y_train)   
    print('training took %fs!' % (time.time() - start_time))
    regr = RandomForestRegressor(max_depth=2, max_features=1)
    regr.fit(X_train, y_train)
    Y_predict_rf = regr.predict(X_test)
    print('predict finished')
    
    modelgbc = classifiers['GBC'](X_train, y_train)   
    print('training took %fs!' % (time.time() - start_time))    
    Y_predict_gbc = modelgbc.predict_proba(X_test)[:,1]
    print('predict finished')
    
    modeldt = classifiers['DT'](X_train, y_train)  
    print('training took %fs!' % (time.time() - start_time))
    regressor = DecisionTreeRegressor(max_depth=2) 
    regressor.fit(X_train, y_train)
    Y_predict_dt = regressor.predict(X_test)
    print('predict finished')


reading training and testing data...
training took 2.251265s!
predict finished
Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   51.6s finished


training took 58.578194s!
predict finished
Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed: 84.1min finished


training took 5182.220682s!
predict finished
training took 5185.944616s!
predict finished


In [ ]:
#cnn test
X_train = cleaned_train[:18141]
X_test = cleaned_train[18141:]
y_train = train_score.score.as_matrix(columns=None).tolist()
X_train = np.asarray(X_train)
X_train = sequence.pad_sequences(X_train, maxlen=maxlen, padding='post')
X_test = sequence.pad_sequences(X_test, maxlen=maxlen, padding='post')


cnnmodel.fit(X_train, y_train, batch_size=128, epochs=8)

Y_predict_cnn = cnnmodel.predict(X_test, verbose=0)
Y_predict_cnn = np.squeeze(Y_predict_cnn, axis=1)

In [16]:
#nn train
X_train = cleaned_train[:18141]
X_test = cleaned_train[18141:]
y_train = train_score.score.as_matrix(columns=None).tolist()
X_array = np.asarray(X_train)
Y_array = np.asarray(y_train)
Xtest_array = np.asarray(X_test) 
import warnings
warnings.filterwarnings("ignore")   

from keras.models import Sequential
from keras.layers import Dense


# fix random seed for reproducibility
np.random.seed(7)

# split into input (X) and output (Y) variables
X = X_array
Y = Y_array
# create model
nnmodel = Sequential()
nnmodel.add(Dense(100, input_dim=b, activation='relu'))
nnmodel.add(Dense(100, activation='relu'))
nnmodel.add(Dense(100, activation='relu'))
nnmodel.add(Dense(100, activation='relu'))
nnmodel.add(Dense(1, activation='sigmoid'))

nnmodel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

nnmodel.fit(X, Y, epochs=80, batch_size=150)

scores = nnmodel.evaluate(X, Y)
print("\n%s: %.2f%%" % (nnmodel.metrics_names[1], scores[1]*100))
Y_predict_nn = nnmodel.predict(Xtest_array, verbose=0)
Y_predict_nn = np.squeeze(Y_predict_nn, axis=1)

Using TensorFlow backend.


Epoch 1/80
18141/18141 [==============================] - 1s 50us/step - loss: 0.6364 - acc: 0.7061
Epoch 2/80
18141/18141 [==============================] - 0s 27us/step - loss: 0.4621 - acc: 0.7879
Epoch 3/80
18141/18141 [==============================] - 1s 29us/step - loss: 0.4435 - acc: 0.8016
Epoch 4/80
18141/18141 [==============================] - 0s 26us/step - loss: 0.4312 - acc: 0.8065
Epoch 5/80
18141/18141 [==============================] - 0s 26us/step - loss: 0.4280 - acc: 0.8099
Epoch 6/80
18141/18141 [==============================] - 0s 27us/step - loss: 0.4216 - acc: 0.8122
Epoch 7/80
18141/18141 [==============================] - 0s 26us/step - loss: 0.4129 - acc: 0.8141
Epoch 8/80
18141/18141 [==============================] - 1s 28us/step - loss: 0.4160 - acc: 0.8144
Epoch 9/80
18141/18141 [==============================] - 1s 32us/step - loss: 0.4084 - acc: 0.8175
Epoch 10/80
18141/18141 [==============================] - 0s 27us/step - loss: 0.4120 - acc: 0.8167

In [18]:
from sklearn.ensemble import VotingClassifier
eclf1 = VotingClassifier(estimators=[('lr', modellr), ('rf', modelrf), ('gbc', modelgbc)], voting='soft')
eclf1 = eclf1.fit(X_train, y_train)
Y_predict_emsemble = eclf1.predict_proba(X_test)[:,1]

In [28]:
Y_predict = []
for i in range(len(Y_predict_rf)):
    Y_predict.append(( Y_predict_lr[i] + Y_predict_gbc[i] + Y_predict_nn[i] ) / 3.0)

In [30]:
temp_score = load_label('submission.csv')
submit_score  = temp_score
submit_score['score'] = Y_predict
submit_score.to_csv('predict_result_lr-gbc-nn.csv')